# Introduction

This tutorial demonstrates how to use the distillation approach based on [Intel® Neural Compressor](https://github.com/intel/neural-compressor) for text-classification.

# Prerequisite

## Install packages

* Follow [installation](https://github.com/intel/intel-extension-for-transformers#installation) to install **intel-extension-for-transformers**. 

In [ ]:
# install model dependency
! pip install accelerate datasets >= 1.1.3 sentencepiece != 0.1.92 protobuf torch >= 1.10 transformers >= 4.12.0 wandb
! pip install -r requirements.txt

## Import packages

In [ ]:
import datasets
import functools
import logging
import os
import numpy as np
import random
import sys
import torch
import transformers
from dataclasses import dataclass, field
from datasets import load_dataset, load_metric
from intel_extension_for_transformers.optimization import (
    metrics,
    DistillationConfig,
    OptimizedModel,
)
from intel_extension_for_transformers.optimization.trainer import NLPTrainer
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from typing import Optional



os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["WANDB_DISABLED"] = "true"


# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.12.0")


task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

logger = logging.getLogger(__name__)

## Define arguments

In [ ]:
# ========== Define arguments =========
@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    Using `HfArgumentParser` we can turn this class
    into argparse arguments to be able to specify them on
    the command line.
    """

    task_name: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the task to train on: " + ", ".join(task_to_keys.keys())},
    )
    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached preprocessed datasets or not."}
    )
    pad_to_max_length: bool = field(
        default=True,
        metadata={
            "help": "Whether to pad all samples to `max_seq_length`. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of prediction examples to this "
            "value if set."
        },
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "A csv or a json file containing the training data."}
    )
    validation_file: Optional[str] = field(
        default=None, metadata={"help": "A csv or a json file containing the validation data."}
    )

    def __post_init__(self):
        if self.task_name is not None:
            self.task_name = self.task_name.lower()
            if self.task_name not in task_to_keys.keys():
                raise ValueError("Unknown task, you should pick one in " + ",".join(task_to_keys.keys()))
        elif self.dataset_name is not None:
            pass
        elif self.train_file is None or self.validation_file is None:
            raise ValueError("Need either a GLUE task, a training/validation file or a dataset name.")
        else:
            train_extension = self.train_file.split(".")[-1]
            assert train_extension in ["csv", "json"], "`train_file` should be a csv or a json file."
            validation_extension = self.validation_file.split(".")[-1]
            assert (
                validation_extension == train_extension
            ), "`validation_file` should have the same extension (csv or json) as `train_file`."


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )


@dataclass
class OptimizationArguments:
    """
    Arguments pertaining to what type of optimization we are going to apply on the model.
    """

    distillation: bool = field(
        default=False,
        metadata={"help": "Whether or not to apply distillation."},
    )
    teacher_model_name_or_path: str = field(
        default=False,
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    metric_name: Optional[str] = field(
        default=None,
        metadata={"help": "Metric used for the tuning strategy."},
    )
    tolerance_mode: Optional[str] = field(
        default="absolute",
        metadata={"help": "Metric tolerance model, expected to be relative or absolute."},
    )
    perf_tol: Optional[float] = field(
        default=0.02,
        metadata={"help": "Performance tolerance when optimizing the model."},
    )
    benchmark: bool = field(
        default=False,
        metadata={"help": "run benchmark."})
    accuracy_only: bool = field(
        default=False,
        metadata={"help":"Whether to only test accuracy for model tuned by Neural Compressor."})

In [ ]:
# We now keep distinct sets of args, for a cleaner separation of concerns.
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments, OptimizationArguments))
if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
    # If we pass only one argument to the script and it's the path to a json file,
    # let's parse it to get our arguments.
    model_args, data_args, training_args, optim_args = parser.parse_json_file(
        json_file=os.path.abspath(sys.argv[1])
    )
else:
    model_args, data_args, training_args, optim_args = parser.parse_args_into_dataclasses()

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

## Download dataset from the hub

In [ ]:
# download the dataset.
raw_datasets = load_dataset("glue", data_args.task_name)
# Labels
label_list = raw_datasets["train"].features["label"].names
num_labels = len(label_list)

## Download fp32 model from the hub

In [ ]:
# Load pretrained model and tokenizer
config = AutoConfig.from_pretrained(
    model_args.model_name_or_path,
    num_labels=num_labels,
    finetuning_task=data_args.task_name,
    revision="main"
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.model_name_or_path,
    use_fast=True,
    revision="main"
)
model = AutoModelForSequenceClassification.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    revision="main"
)

## Preprocessing the dataset

In [ ]:
# Preprocessing the raw_datasets
sentence1_key, sentence2_key = task_to_keys[data_args.task_name]
# Padding strategy
padding = False
# Some models have set the order of the labels to use, so let's make sure we do use it.
label_to_id = None
if model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id:
    # Some have all caps in their config, some don't.
    label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
    if list(sorted(label_name_to_id.keys())) == list(sorted(label_list)):
        label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
    else:
        logger.warning(
            f"Your model seems to have been trained with labels, but they don't match the dataset: "
            f"model labels: {list(sorted(label_name_to_id.keys()))}, dataset labels: {list(sorted(label_list))}.\n"
            f"Ignoring the model labels as a result."
        )
if label_to_id is not None:
    model.config.label2id = label_to_id
    model.config.id2label = {id: label for label, id in config.label2id.items()}
max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)

def preprocess_function(examples, tokenizer=tokenizer):
    # Tokenize the texts
    args = (
        (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
    )
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    return result

with training_args.main_process_first(desc="dataset map pre-processing"):
    raw_datasets = raw_datasets.map(
        preprocess_function, batched=True, load_from_cache_file=not data_args.overwrite_cache
    )

if training_args.do_train:
    if "train" not in raw_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = raw_datasets["train"]

if training_args.do_eval:
    if "validation" not in raw_datasets and "validation_matched" not in raw_datasets:
        raise ValueError("--do_eval requires a validation dataset")
    eval_dataset = raw_datasets["validation_matched" if data_args.task_name == "mnli" else "validation"]

# Log a few random samples from the training set:
if training_args.do_train:
    for index in random.sample(range(len(train_dataset)), 3):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

# Get the metric function
metric = load_metric("glue", data_args.task_name)

metric_name = "eval_accuracy"

# You can define your custom compute_metrics function. It takes an `EvalPrediction` object (a namedtuple with
# predictions and label_ids field) and has to return a dictionary string to float.
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds =  np.argmax(preds, axis=1)
    if data_args.task_name is not None:
        result = metric.compute(predictions=preds, references=p.label_ids)
        if len(result) > 1:
            result["combined_score"] = np.mean(list(result.values())).item()
        return result
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

# Data collator will default to DataCollatorWithPadding, so we change it if we already did the padding.
data_collator = None

# Distillation & Benchmark

## Model Distillation

In [ ]:
class BertModelforLogitsOutputOnly(torch.nn.Module):
    def __init__(self, model):
        super(BertModelforLogitsOutputOnly, self).__init__()
        self.model = model
    def forward(self, *args, **kwargs):
        output = self.model(*args, **kwargs)
        return output['logits']

teacher_config = AutoConfig.from_pretrained(optim_args.teacher_model_name_or_path, \
                    num_labels=num_labels, finetuning_task=data_args.task_name)
teacher_tokenizer = AutoTokenizer.from_pretrained(optim_args.teacher_model_name_or_path, \
                    use_fast=model_args.use_fast_tokenizer)
teacher_model = AutoModelForSequenceClassification.from_pretrained(
    optim_args.teacher_model_name_or_path,
    from_tf=bool(".ckpt" in optim_args.teacher_model_name_or_path),
    config=teacher_config,
)
teacher_model.to(training_args.device)

# prepare datasets for teacher model
teacher_processed_datasets = raw_datasets.map(
    functools.partial(preprocess_function, tokenizer=teacher_tokenizer), 
    batched=True, remove_columns=raw_datasets["train"].column_names
)
teacher_train_dataset = teacher_processed_datasets["train"]
teacher_train_dataset = teacher_train_dataset.select(range(data_args.max_train_samples))
teacher_eval_dataset = teacher_processed_datasets["validation_matched" \
                            if data_args.task_name == "mnli" else "validation"]
teacher_eval_dataset = teacher_eval_dataset.select(range(data_args.max_eval_samples))
    
# get logits of teacher model
def dict_tensor_to_model_device(batch, model):
    device = next(model.parameters()).device
    for k in batch:
        batch[k] = batch[k].to(device)

def get_logits(teacher_model, train_dataset, teacher_train_dataset):
    logger.info("***** Getting logits of teacher model *****")
    logger.info(f"  Num examples = {len(train_dataset) }")
    teacher_model.eval()
    npy_file = os.path.join(os.path.dirname(os.path.abspath(__file__)),
        '{}.{}.npy'.format(data_args.task_name, 
                            optim_args.teacher_model_name_or_path.replace('/', '.')))
    if os.path.exists(npy_file):
        teacher_logits = [x for x in np.load(npy_file)]
    else:
        sampler = None
        if training_args.world_size > 1:
            from transformers.trainer_pt_utils import ShardSampler
            sampler = ShardSampler(
                teacher_train_dataset,
                batch_size=training_args.per_device_eval_batch_size,
                num_processes=training_args.world_size,
                process_index=training_args.process_index,
            )
            teacher_model = torch.nn.parallel.DistributedDataParallel(
                teacher_model,
                device_ids=[training_args.local_rank] \
                    if training_args._n_gpu != 0 else None,
                output_device=training_args.local_rank \
                    if training_args._n_gpu != 0 else None,
            )
        train_dataloader = DataLoader(teacher_train_dataset, 
                                        collate_fn=data_collator,
                                        sampler=sampler,
                                        batch_size=training_args.per_device_eval_batch_size)
        train_dataloader = tqdm(train_dataloader, desc="Evaluating")
        teacher_logits = []
        for step, batch in enumerate(train_dataloader):
            dict_tensor_to_model_device(batch, teacher_model)
            outputs = teacher_model(**batch)
            if training_args.world_size > 1:
                outputs_list = [None for i in range(training_args.world_size)]
                torch.distributed.all_gather_object(outputs_list, outputs)
                outputs = torch.concat(outputs_list, dim=0)
            teacher_logits += [x for x in outputs.cpu().numpy()]
        if training_args.world_size > 1:
            teacher_logits = teacher_logits[:len(teacher_train_dataset)]
        if training_args.local_rank in [-1, 0]:
            np.save(npy_file, np.array(teacher_logits))
    return train_dataset.add_column('teacher_logits', teacher_logits)

with torch.no_grad():
    train_dataset = get_logits(BertModelforLogitsOutputOnly(teacher_model), train_dataset, teacher_train_dataset)
        
para_counter = lambda model:sum(p.numel() for p in model.parameters())
logger.info("***** Number of teacher model parameters: {:.2f}M *****".format(\
            para_counter(teacher_model)/10**6))
logger.info("***** Number of student model parameters: {:.2f}M *****".format(\
            para_counter(model)/10**6))

In [ ]:
# Set seed before initializing model.
set_seed(training_args.seed)
# Initialize our Trainer
trainer = NLPTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
# distillation
if not training_args.do_eval:
    raise ValueError("do_eval must be set to True for distillation.")

tune_metric = metrics.Metric(name=metric_name)
distillation_conf = DistillationConfig(metrics=tune_metric)
model = trainer.distill(
    distillation_config=distillation_conf, teacher_model=teacher_model
)
trainer.save_model(training_args.output_dir)

## Run Benchmark after Distillation

In [ ]:
# Load the model obtained after Intel Neural Compressor (INC) quantization
model = OptimizedModel.from_pretrained(
    training_args.output_dir,
)
model.eval()
trainer.model = model
results = trainer.evaluate()
logger.info("metrics keys: {}".format(results.keys()))
bert_task_acc_keys = ['eval_f1', 'eval_accuracy', 'eval_matthews_correlation',
                        'eval_pearson', 'eval_mcc', 'eval_spearmanr']

for key in bert_task_acc_keys:
    if key in results.keys():
        ret = True
        throughput = results.get("eval_samples_per_second")
        print('Batch size = ', training_args.per_device_eval_batch_size)
        print("Finally Eval {} Accuracy: {}".format(key, results[key]))
        print("Latency: {:.5f} ms".format(1000 / throughput))
        print("Throughput: {:.5f} samples/sec".format(throughput))